In [ ]:
# %pip install langchain langchain-openai langchain-Groq langchain-community pypdf chromadb

In [ ]:
# pip install arxiv langchain langchain-openai langchain-Groq langchain-community pypdf chromadb

In [ ]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from google.colab import userdata
import os
import os
import getpass
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['GROQ_API_KEY']=userdata.get('gorq_api_key')
os.environ['LANGSMITH_API_KEY']=userdata.get('lang_api_key')
os.environ['OPENAI_API_KEY']=userdata.get('open_api_key')
os.environ["LANGCHAIN_PROJECT"] = "lang_api_pro"
os.environ["LANGSMITH_PROJECT"]="test"


#Knowledge Base

##1- Document Loading

In [ ]:
import arxiv
import os

# Create the 'data' directory if it doesn't exist
os.makedirs('./data', exist_ok=True)


search = arxiv.Search(
  query = "Retrieval Augmented Generation",
  max_results = 10,
  sort_by = arxiv.SortCriterion.Relevance
)


for result in search.results():
    file_path = result.download_pdf(dirpath="./data")
    print(f"تم تحميل البحث: {result.title} -> {file_path}")

/tmp/ipython-input-3114970597.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


تم تحميل البحث: AR-RAG: Autoregressive Retrieval Augmentation for Image Generation -> ./data/2506.06962v3.AR_RAG__Autoregressive_Retrieval_Augmentation_for_Image_Generation.pdf
تم تحميل البحث: Intelligent Interaction Strategies for Context-Aware Cognitive Augmentation -> ./data/2504.13684v1.Intelligent_Interaction_Strategies_for_Context_Aware_Cognitive_Augmentation.pdf
تم تحميل البحث: Factually: Exploring Wearable Fact-Checking for Augmented Truth Discernment -> ./data/2504.17204v1.Factually__Exploring_Wearable_Fact_Checking_for_Augmented_Truth_Discernment.pdf
تم تحميل البحث: Designing AI Systems that Augment Human Performed vs. Demonstrated Critical Thinking -> ./data/2504.14689v1.Designing_AI_Systems_that_Augment_Human_Performed_vs__Demonstrated_Critical_Thinking.pdf
تم تحميل البحث: Automated Literature Review Using NLP Techniques and LLM-Based Retrieval-Augmented Generation -> ./data/2411.18583v1.Automated_Literature_Review_Using_NLP_Techniques_and_LLM_Based_Retrieval_Augmented_Gene

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2- Splitter

In [ ]:
def load_and_split_pdfs(data_path="./data"):
    docs = []
    for file in os.listdir(data_path):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(data_path, file))
            docs.extend(loader.load())

    # split text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(docs)
    print(f" Total Chunks: {len(chunks)}")
    return chunks
chunks = load_and_split_pdfs()

 Total Chunks: 1336


In [ ]:

chunks


[Document(metadata={'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'creator': 'LaTeX with acmart 2024/12/28 v2.12 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX', 'creationdate': '2025-04-21T00:40:31+00:00', 'moddate': '2025-04-21T00:40:31+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'Intelligent Interaction Strategies for Context-Aware Cognitive Augmentation', 'trapped': '/False', 'source': './data/2504.13684v1.Intelligent_Interaction_Strategies_for_Context_Aware_Cognitive_Augmentation.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Intelligent Interaction Strategies for Context-Aware Cognitive Augmentation\nXIANGRONG (DANIEL) ZHU, YUAN XU, and TIANJIAN LIU, The Hong Kong University of Science and\nTechnology (Guangzhou), China\nJINGWEI SUN and YU ZH

In [ ]:
chunks[0].metadata


{'producer': 'pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'creator': 'LaTeX with acmart 2024/12/28 v2.12 Typesetting articles for the Association for Computing Machinery and hyperref 2023-04-22 v7.00x Hypertext links for LaTeX',
 'creationdate': '2025-04-21T00:40:31+00:00',
 'moddate': '2025-04-21T00:40:31+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5',
 'subject': '',
 'title': 'Intelligent Interaction Strategies for Context-Aware Cognitive Augmentation',
 'trapped': '/False',
 'source': './data/2504.13684v1.Intelligent_Interaction_Strategies_for_Context_Aware_Cognitive_Augmentation.pdf',
 'total_pages': 8,
 'page': 0,
 'page_label': '1'}

In [ ]:
chunks[0].page_content

'Intelligent Interaction Strategies for Context-Aware Cognitive Augmentation\nXIANGRONG (DANIEL) ZHU, YUAN XU, and TIANJIAN LIU, The Hong Kong University of Science and\nTechnology (Guangzhou), China\nJINGWEI SUN and YU ZHANG, Lenovo Research, China\nXIN TONG†, The Hong Kong University of Science and Technology (Guangzhou) and The Hong Kong University of\nScience and Technology, China'

## 3- Embeddings, Vector Store and Retrieval

In [ ]:
# pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./db_free"
)
retriever = vector_db.as_retriever()
print("✅ Vector Database is ready and saved locally!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Vector Database is ready and saved locally!


In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})


In [ ]:
result = retriever.invoke("what is RAG")
result

[Document(metadata={'author': 'Dean Wampler; Dave Nielson; Alireza Seddighi', 'doi': 'https://doi.org/10.48550/arXiv.2601.05264', 'page_label': '5', 'producer': 'pikepdf 8.15.1', 'total_pages': 86, 'creator': 'arXiv GenPDF (tex2pdf:57610bf)', 'license': 'http://creativecommons.org/licenses/by/4.0/', 'source': './data/2601.05264v1.Engineering_the_RAG_Stack__A_Comprehensive_Review_of_the_Architecture_and_Trust_Frameworks_for_Retrieval_Augmented_Generation_Systems.pdf', 'creationdate': '', 'arxivid': 'https://arxiv.org/abs/2601.05264v1', 'title': 'Engineering the RAG Stack: A Comprehensive Review of the Architecture and Trust Frameworks for Retrieval-Augmented Generation Systems', 'page': 4, 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1'}, page_content='1.4 Article Contributions and Research Foundation\nBy conducting a comprehensive, technically rigorous, and critical assessment\nof the field, this survey en

#Chabot

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are an Expert with 10 years experiance in RAG feild answer the questions, based on the context provided below only. Provide source metadata from context after your answer"),
        HumanMessagePromptTemplate.from_template("""
        Context:
        {context}

        Question:
        {question}
        """)
        ]
    )
# llm = ChatGroq(model="openai/gpt-oss-120b")
# Parser = StrOutputParser()
llm=ChatOpenAI(model="gpt-5-nano")
Parser = StrOutputParser()

In [ ]:

open_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
# Query the system
response = open_chain.invoke("What is the specific policy/procedure for RAG according to our uploaded documents, and how does it differ from standard industry practices?")
print(response)

Answer:

Specific policy/procedure for RAG in the uploaded document:
- The document prescribes a methodological-transparency policy for evaluating RAG systems. Key elements include:
  - Methodological Transparency: Evaluate how results are interpreted, ensure the experimental setup is clearly described, provide implementation details, and maintain high-quality documentation. Studies that provide adequate detail to enable replication and validation are prioritized.
  - Relevance and Contribution: Assess the direct relevance to RAG systems, the contribution to architectural understanding, and practical applicability.

- Practical you-should-do steps (as implied by the text):
  1) Clearly state how results should be interpreted.
  2) Document the experimental setup in detail.
  3) Provide sufficient implementation details to enable replication.
  4) Ensure documentation quality supports validation by others.
  5) Evaluate whether work adds value to RAG architecture and has practical utili

In [ ]:
response=open_chain.invoke("what is best thing in RAG system ?")
print(response)

Based on the provided context, the best thing in a RAG system is to adopt a comprehensive, unified architectural framework that provides a solid taxonomy and integrates trust and safety. This foundation helps guide design toward scalable, reliable, and trustworthy production systems by:

- Establishing a systematic architectural taxonomy of RAG designs (retrieval logic, fusion topology) to unify the field.
- Addressing scalability and deployment through a structured understanding of designs, complexity, and performance trade-offs.
- Integrating trust calibration and safety mechanisms directly into the architecture.
- Highlighting patterns and anti-patterns to avoid common reliability pitfalls.
- Focusing on the evolution from canonical to agentic RAG systems with an eye toward production-readiness.

Source: The article “Engineering the RAG Stack: A Comprehensive Review of the Architecture and Trust Frameworks for Retrieval-Augmented Generation Systems” by Dean Wampler; Dave Nielson; Al

In [ ]:
def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Define the RAG chain
rag_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | prompt_template
   | llm
   | StrOutputParser()
)

In [ ]:
# Query the system
response = rag_chain.invoke("What does our documentation say about machine learning?")
print(response)

Our documentation notes that **machine‑learning techniques are being applied to automate literature‑review tasks**. Jaspers et al. examined a range of ML approaches, summarising their advantages and disadvantages and describing the automation workflow in detail—though they also point out that the work has not yet been validated across many domains or provided deep practical insights【Context】.  

In addition, the documentation references recent advances in **retrieval‑augmented language modeling**, where large language models are improved by pulling information from “trillions of tokens” (Cassirer et al., ICML 2022)【Context】, and it highlights resources on **self‑RAG (retrieval‑augmented generation)** for prompting (Prompting 2024)【Context】.  

Finally, a 2024 market report (“The State of Generative AI in the Enterprise”) is cited as a source for broader enterprise‑level trends in generative AI, which includes machine‑learning‑driven generative models【Context】.  



**Sources**  
- Jasp

In [ ]:
prompt_trans = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("You are an Expert with 10 years experiance in RAG feild, and You are Egyptian Translator, your job is tranlsate context from English to Egyptian Dialect."),
        HumanMessagePromptTemplate.from_template("""
        Context:
        {context}
        """)
        ]
    )
llm_trans = ChatGroq(model="openai/gpt-oss-120b")
Parser_trans = StrOutputParser()

In [ ]:
trans_chain = prompt_trans | llm_trans | Parser_trans

In [ ]:
ar_response = trans_chain.invoke({"context":response})

In [ ]:
print(ar_response)

**النص المترجم للعامية المصرية**

التوثيق بتاعنا بيقول إن **تقنيات التعلم الآلي** بتتطبق دلوقتي عشان تُ automate (تلقائي) مهام مراجعة الأدبيات. جaspers وآخرين درسوا مجموعة من أساليب الـ ML، لخصوا مميزات وعيوب كل طريقة ووصفوا سير العمل الأوتوماتيكي بالتفصيل — لكن كمان أشاروا إن الشغل ده لسه ما اتثبتش على نطاق واسع في مجالات كتير ولا قدم رؤى عملية عميقة.

كمان التوثيق بيشير للتطورات الحديثة في **نماذج اللغة المدعومة بالاسترجاع**، اللي فيها نماذج اللغة الكبيرة بتتحسّن لما تجيب معلومات من “تريليونات من التوكنز” (Cassirer وآخرين، ICML 2022)، وبيبرز موارد عن **self‑RAG (استرجاع‑مُعزز‑التوليد)** للتوجيه (Prompting 2024).

وأخيرًا، تقرير سوق 2024 بعنوان “**حالة الذكاء الاصطناعي التوليدي في الشركات**” اتذكر كمصدر للاتجاهات العامة على مستوى الشركات في الذكاء الاصطناعي التوليدي، واللي بيشمل نماذج توليدية مدفوعة بالتعلم الآلي.

---

**المصادر**  
- Jaspers et al. (التعلم الآلي لمراجعات الأدبيات الأوتوماتيكية) – مقتطف من السياق.  
- Cassirer et al. “تحسين نماذج اللغة بالاسترجاع من تريليونات من التو

#### Streamlit

In [ ]:
# Keep chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Show history
for message in st.session_state.messages:
    st.chat_message(message["role"]).markdown(message["content"])

# User input
user_input = st.chat_input("This is AI Assistant for RAG information, How can I help you?")
if user_input:
    # Show user message
    st.chat_message("user").markdown(user_input)
    # Save User message
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Show Streamed response
    with st.chat_message("ai"):
        streamed_text = st.write_stream(open_chain.stream(user_input))

    # Save AI reply
    st.session_state.messages.append({"role": "ai", "content": streamed_text})